[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/HyperCoast/blob/main/docs/examples/chla_predict.ipynb)

# Chlorophyll-a prediction with Deep Learning

## Install packages

Uncomment the following cell to install the required packages.

In [ ]:
# %pip install "hypercoast[all]"

## Import libraries

In [ ]:
import torch
import hypercoast
from hypercoast.chla import *

## Download sample data

In [ ]:
chla_data_url = "https://github.com/opengeos/datasets/releases/download/hypercoast/chla_test_data.zip"
pace_data_url = "https://github.com/opengeos/datasets/releases/download/hypercoast/PACE_OCI.20241024T182127.L2.OC_AOP.V2_0.NRT.nc"

In [ ]:
hypercoast.download_file(chla_data_url, quiet=True)

In [ ]:
pace_filepath = hypercoast.download_file(pace_data_url, quiet=True)

## Train the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load the training dataset
train_real_dl, test_real_dl, input_dim, output_dim = load_real_data(
    "data/Chl_RC_PACE.csv", "data/Rrs_RC_PACE.csv"
)
# Load the validation dataset
test_real_Sep, _, _ = load_real_test(
    "data/Chl_RC_PACE_Sep.csv", "data/Rrs_RC_PACE_Sep.csv"
)
# Model output path.
save_dir = "model/VAE_Chla_PACE"
os.makedirs(save_dir, exist_ok=True)

# Create the VAE model and optimizer
model = VAE(input_dim, output_dim).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)

In [ ]:
best_model_path = "model/vae_trans_model_best_Chl_PACE.pth"

In [ ]:
train(model, train_real_dl, epochs=400, best_model_path=best_model_path)
# Load the optimal model
model.load_state_dict(torch.load(best_model_path, map_location=device))

predictions, actuals = evaluate(model, test_real_dl)

predictions_Sep, actuals_Sep = evaluate(model, test_real_Sep)

save_to_csv(predictions, os.path.join(save_dir, "predictions.csv"))
save_to_csv(actuals, os.path.join(save_dir, "actuals.csv"))

In [ ]:
plot_results(predictions, actuals, save_dir, mode="test")

![image](https://github.com/user-attachments/assets/8766a2a9-8bdf-4348-a544-7115cfb95f63)

In [ ]:
plot_results(predictions_Sep, actuals_Sep, save_dir, mode="Sep")

![image](https://github.com/user-attachments/assets/2e16bbc5-9b8c-46ce-88f5-9fb6187fe5af)

## Predict chlorophyll-a concentration

In [ ]:
chla_data_file = pace_filepath.replace(".nc", ".npy")

In [ ]:
chla_predict(pace_filepath, best_model_path, chla_data_file, device)

In [ ]:
geotiff_file = chla_data_file.replace(".npy", ".tif")
npy_to_geotiff(chla_data_file, geotiff_file, max_distance_m=5000)

## Visualize the results

In [ ]:
rgb_image_tif_file = "data/snapshot-2024-08-10T00_00_00Z.tif"
output_file = "20241024-2.png"
title = "PACE Chla Prediction"
figsize = (12, 8)
cmap = "jet"

In [ ]:
chla_viz(rgb_image_tif_file, chla_data_file, output_file, title, figsize, cmap)

![image](https://github.com/user-attachments/assets/93944a8d-8784-441a-9a51-9050780eb22e)

In [ ]:
m = hypercoast.Map()
m.add_basemap("Esri.WorldImagery")
m.add_raster(
    geotiff_file, colormap="jet", nodata=-9999.0, vmin=0, vmax=35, layer_name="Chla"
)
m